# Image Classification

First I will download the data set from kaggle and drag and drop it into the colab file explorer.

The data is zipped so I need to unzip them.

In [ ]:
!unzip /content/Archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: seg_train/seg_train/mountain/16164.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._16164.jpg  
  inflating: seg_train/seg_train/mountain/3358.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._3358.jpg  
  inflating: seg_train/seg_train/mountain/2720.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._2720.jpg  
  inflating: seg_train/seg_train/mountain/14029.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._14029.jpg  
  inflating: seg_train/seg_train/mountain/14997.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._14997.jpg  
  inflating: seg_train/seg_train/mountain/1567.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._1567.jpg  
  inflating: seg_train/seg_train/mountain/11637.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._11637.jpg  
  inflating: seg_train/seg_train/mountain/15445.jpg  
  inflating: __MACOSX/seg_train/seg_train/mountain/._15445.jpg  
  i

Inside the zipped file there is, seg_pred, seg_train , and seg_test. I will assign them to variables so that I can access them in the code later on.

In [ ]:
seg_pred = '/content/seg_pred/seg_pred'
seg_train = '/content/seg_train/seg_train'
seg_test = '/content/seg_test/seg_test'

Now I need to transform the images into tensor data. I will use the ImageDataGenerator to turn image files on disk into batches of pre-processed tensors. My first idea was to use binary as the mode, but that would be used if I were classifying the data into 2 different categories. Instead, we have 6 different categories to choose from, so I will use categorical class_mode.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        seg_train,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # This is used to ger categorical labels 
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        seg_pred,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        seg_test,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')




Found 14034 images belonging to 6 classes.
Found 0 images belonging to 0 classes.
Found 3000 images belonging to 6 classes.


I am going to construct a fully connected model. I will make sure to add 6 as the last layer to match the shape of our data which has 6 categories. 

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 64)        36928     
                                                                 
 flatten_3 (Flatten)         (None, 73984)            

Now I will need to compile the model. I will use. Since I am working with 6 categories of data, I will set the loss to categorical_crossentropy.

In [ ]:
from keras import optimizers
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Now we will fit our model and yield batches of inputs and targets over 10 epochs.

In [ ]:
history = model.fit(
          train_generator,
          steps_per_epoch=100,
          epochs=10,
          validation_data=validation_generator,
          validation_steps=50)

Epoch 1/10
100/100 [==============================] - 5s 40ms/step - loss: 1.4221 - accuracy: 0.4285
Epoch 2/10
100/100 [==============================] - 3s 30ms/step - loss: 1.1200 - accuracy: 0.5730
Epoch 3/10
100/100 [==============================] - 3s 29ms/step - loss: 1.0216 - accuracy: 0.6005
Epoch 4/10
100/100 [==============================] - 4s 40ms/step - loss: 0.9515 - accuracy: 0.6415
Epoch 5/10
100/100 [==============================] - 3s 31ms/step - loss: 0.9294 - accuracy: 0.6535
Epoch 6/10
100/100 [==============================] - 3s 30ms/step - loss: 0.9010 - accuracy: 0.6650
Epoch 7/10
100/100 [==============================] - 4s 42ms/step - loss: 0.8749 - accuracy: 0.6745
Epoch 8/10
100/100 [==============================] - 3s 32ms/step - loss: 0.8581 - accuracy: 0.6770
Epoch 9/10
100/100 [==============================] - 3s 30ms/step - loss: 0.8329 - accuracy: 0.6955
Epoch 10/10
100/100 [==============================] - 4s 40ms/step - loss: 0.7738 - accura

Now I will evaluate the test data to measure the accuracy of our classifier.

In [ ]:
scoreSeg = model.evaluate(test_generator)
print("Accuracy: ", scoreSeg[1])

150/150 [==============================] - 5s 32ms/step - loss: 0.8963 - accuracy: 0.6677
Accuracy:  0.6676666736602783
